In [1]:
# Install core libraries
%pip install --upgrade pip
%pip install pandas requests tqdm python-dateutil gitpython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pathlib import Path
import pandas as pd

CSV_URL = "https://gitlab.com/exploit-database/exploitdb/-/raw/main/files_exploits.csv"
GIT_URL = "https://gitlab.com/exploit-database/exploitdb.git"

OUT_CSV = "exploitdb_extracted.csv"
WORKDIR = Path.cwd()
print("Working directory:", WORKDIR)

Working directory: /Users/adanikamal/Documents/FYP_Project/scripts


In [5]:
import requests, time
from tqdm import tqdm

def download_file(url, output, retries=3):
    for i in range(1, retries+1):
        try:
            with requests.get(url, stream=True, timeout=30) as r:
                r.raise_for_status()
                total = int(r.headers.get('content-length', 0))
                with open(output, 'wb') as f, tqdm(total=total, unit='B', unit_scale=True, desc="Downloading") as bar:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
                            bar.update(len(chunk))
            return True
        except Exception as e:
            print(f"Attempt {i} failed:", e)
            time.sleep(2*i)
    return False

csv_path = WORKDIR / "files_exploits.csv"
ok = download_file(CSV_URL, csv_path)
print("Download OK?", ok)

Downloading: 100%|██████████████████████████████| 10.1M/10.1M [00:01<00:00, 5.48MB/s]

Download OK? True


In [15]:
import pandas as pd
from pathlib import Path

raw_path = Path("files_exploits.csv")   # change if your raw file is elsewhere
out_path = Path("exploitdb_extracted_fixed_date.csv")

# load
df = pd.read_csv(raw_path, dtype=str, keep_default_na=False)
print("Raw shape:", df.shape)

# normalize empty strings -> NaN for easier handling
df = df.replace({"": pd.NA})

# prefer date_published, else date_added
if "date_published" in df.columns:
    date_series = df["date_published"]
elif "published" in df.columns:
    date_series = df["published"]
else:
    date_series = pd.Series([pd.NA]*len(df))

if "date_added" in df.columns:
    date_series = date_series.fillna(df["date_added"])

# parse to datetime where possible and normalize to YYYY-MM-DD
date_parsed = pd.to_datetime(date_series, errors="coerce")
date_norm = date_parsed.dt.strftime("%Y-%m-%d").fillna(pd.NA)

# build output DataFrame using canonical names you requested
out = pd.DataFrame({
    "title": df.get("description", df.columns[0]),  # description column is title
    "cve_id": df.get("cve", pd.NA),                 # may be missing; extracted later if needed
    "date": date_norm,
    "type": df.get("type", pd.NA),
    "platform": df.get("platform", pd.NA),
    "vulnerable_application": df["application_url"].notna().map({True:"yes", False:"no"}) if "application_url" in df.columns else "no"
})

# if a CVE column does not exist, try to extract CVE pattern from description
if out["cve_id"].isna().all():
    combined = df.astype(str).agg(" ".join, axis=1)
    out["cve_id"] = combined.str.extract(r'(CVE-\d{4}-\d{4,7})', expand=False)

# save
out.to_csv(out_path, index=False)
print("Saved:", out_path)
print("Output shape:", out.shape)

# quick verification
print("Count rows with non-empty date:", out["date"].notna().sum())
display(out.head(10))

Raw shape: (46922, 17)
Saved: exploitdb_extracted_fixed_date.csv
Output shape: (46922, 6)
Count rows with non-empty date: 46922


,title,cve_id,date,type,platform,vulnerable_application
0,AIX Calendar Manager Service Daemon (rpc.cmsd)...,CVE-2009-3699,2010-11-11,dos,aix,no
1,AppleShare IP Mail Server 5.0.3 - Buffer Overflow,CVE-1999-1015,1999-10-15,dos,aix,no
2,BSDI 4.0 tcpmux / inetd - Crash,NaN,1998-04-07,dos,aix,no
3,Flussonic Media Server 4.1.25 < 4.3.3 - Arbitr...,NaN,2014-07-01,dos,aix,no
4,IBM AIX 4.3.1 - 'adb' Denial of Service,NaN,1999-07-12,dos,aix,no
5,IBM AIX 4.3.3/5.1/5.2 - 'libIM' Buffer Overflow,CVE-2003-0087,2003-02-12,dos,aix,no
6,IBM AIX 5.x - 'Invscout' Local Buffer Overflow,NaN,2005-06-09,dos,aix,no
7,PHP Hosting Directory 2.0 - Database Disclosure,NaN,2010-10-16,dos,aix,yes
8,PHP Stock Management System 1.02 - Multiple Vu...,NaN,2014-09-09,dos,aix,no
9,PointDev IDEAL Migration - Buffer Overflow (Me...,CVE-2009-4265,2010-09-25,dos,aix,yes


In [10]:
out.to_csv(OUT_CSV, index=False)
print("Saved cleaned CSV as:", OUT_CSV)

Saved cleaned CSV as: exploitdb_extracted.csv


In [13]:
#Print header, row count, and non-empty counts per column
import pandas as pd
from pathlib import Path

raw = Path("files_exploits.csv")  # change if different path
df = pd.read_csv(raw, dtype=str, keep_default_na=False)

print("Shape:", df.shape)
print("Columns:")
for i,c in enumerate(df.columns):
    nonempty = (df[c].astype(str).str.strip() != "").sum()
    print(f" {i+1:02d}. {c!r}  — non-empty: {nonempty}")
    
# show first 8 rows (all columns) to eyeball where date might be
display(df.head(8))

Shape: (46922, 17)
Columns:
 01. 'id'  — non-empty: 46922
 02. 'file'  — non-empty: 46922
 03. 'description'  — non-empty: 46922
 04. 'date_published'  — non-empty: 46922
 05. 'author'  — non-empty: 46922
 06. 'type'  — non-empty: 46922
 07. 'platform'  — non-empty: 46922
 08. 'port'  — non-empty: 3355
 09. 'date_added'  — non-empty: 46922
 10. 'date_updated'  — non-empty: 39828
 11. 'verified'  — non-empty: 46922
 12. 'codes'  — non-empty: 32324
 13. 'tags'  — non-empty: 5159
 14. 'aliases'  — non-empty: 484
 15. 'screenshot_url'  — non-empty: 1458
 16. 'application_url'  — non-empty: 8335
 17. 'source_url'  — non-empty: 19522


,id,file,description,date_published,author,type,platform,port,date_added,date_updated,verified,codes,tags,aliases,screenshot_url,application_url,source_url
0,16929,exploits/aix/dos/16929.rb,AIX Calendar Manager Service Daemon (rpc.cmsd)...,2010-11-11,Metasploit,dos,aix,,2010-11-11,2011-03-06,1,CVE-2009-3699;OSVDB-58726,Metasploit Framework (MSF),,,,http://aix.software.ibm.com/aix/efixes/securit...
1,19046,exploits/aix/dos/19046.txt,AppleShare IP Mail Server 5.0.3 - Buffer Overflow,1999-10-15,Chris Wedgwood,dos,aix,,1999-10-15,2014-01-02,1,CVE-1999-1015;OSVDB-5970,,,,,https://www.securityfocus.com/bid/61/info
2,19049,exploits/aix/dos/19049.txt,BSDI 4.0 tcpmux / inetd - Crash,1998-04-07,Mark Schaefer,dos,aix,,1998-04-07,2014-01-02,1,OSVDB-82889,,,,,https://www.securityfocus.com/bid/66/info
3,33943,exploits/aix/dos/33943.txt,Flussonic Media Server 4.1.25 < 4.3.3 - Arbitr...,2014-07-01,BGA Security,dos,aix,8080,2014-07-01,2014-07-01,0,OSVDB-108610;OSVDB-108609,,,,,
4,19418,exploits/aix/dos/19418.txt,IBM AIX 4.3.1 - 'adb' Denial of Service,1999-07-12,GZ Apple,dos,aix,,1999-07-12,2017-11-15,1,OSVDB-83455,,,,,https://www.securityfocus.com/bid/520/info
5,22249,exploits/aix/dos/22249.txt,IBM AIX 4.3.3/5.1/5.2 - 'libIM' Buffer Overflow,2003-02-12,Euan Briggs,dos,aix,,2003-02-12,2012-10-25,1,CVE-2003-0087;OSVDB-7996,,,,,https://www.securityfocus.com/bid/6840/info
6,25807,exploits/aix/dos/25807.txt,IBM AIX 5.x - 'Invscout' Local Buffer Overflow,2005-06-09,Computer Academic Underground,dos,aix,,2005-06-09,2013-05-29,1,,,,,,https://www.securityfocus.com/bid/13909/info
7,15264,exploits/aix/dos/15264.py,PHP Hosting Directory 2.0 - Database Disclosure,2010-10-16,ZoRLu,dos,aix,,2010-10-16,2010-10-16,1,,,,,http://www.exploit-db.comphphost_directory.zip,


In [30]:
import pandas as pd

df = pd.read_csv("/Users/adanikamal/Documents/FYP_Project/scripts/exploitdb_extracted_fixed_date.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46922 entries, 0 to 46921
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   46922 non-null  object
 1   cve_id                  27229 non-null  object
 2   date                    46922 non-null  object
 3   type                    46922 non-null  object
 4   platform                46922 non-null  object
 5   vulnerable_application  46922 non-null  object
dtypes: object(6)
memory usage: 2.1+ MB


In [38]:
import pandas as pd

df = pd.read_csv("/Users/adanikamal/Documents/FYP_Project/scripts/exploitdb_extracted_fixed_date.csv")
df.shape

(46922, 6)